In [1]:
from nanorsm_parallel_recovered import *

/nsls2/conda/envs/2025-2.2-py312-tiled/lib/python3.12/site-packages/databroker/assets/base_registry.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Kafka server not available, Databroker will not be able to upload datum


In [2]:
# load elemental image data, align it with pystackreg and generate a tranform matrix

data_path = '//data//users//2025Q2//Marschilok_2025Q2//xrf//'
# sid_list = np.concatenate((np.sort(get_sid_list(['219698-219728'],3))[::-1],get_sid_list(['219620-219692'],3)))
# sid_list = get_sid_list(['219528-219588'],3)
sid_list = get_sid_list(['346726-346736'],2)

elem = 'Ni_K'
file_list = [
    f"{data_path}output_tiff_scan2D_{sid}//detsum_{elem}_norm.tiff"
    for sid in sid_list
]
im_stack = load_ims(file_list)
num_frame,im_row,im_col = np.shape(im_stack)
im_stack_aligned, trans_matrix = align_im_stack(im_stack) # use pystackreg

#im_stack = np.delete(im_stack,0,axis=0)
#trans_matrix = np.delete(trans_matrix,0,axis=0)

im_stack_test = interp_sub_pix(im_stack,trans_matrix) # verify the alignment is done correctly

print(f"image stack size ={im_stack_test.shape}")
# tifffile.imwrite(''.join([data_path, 'original_rsm_100x100_nm//stack_aligned_', elem, '_100x100_nm.tif']),im_stack_test.astype(np.float32),imagej=True)

Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

image stack size =(6, 60, 60)


In [3]:
# if there are more elements to be aligned and stacked
elem_list = ['Ni_K', 'Mn_K','Co_K']
# sid_list = get_sid_list(['219792-219888'], 3)

stack = []
for i, elem in enumerate(elem_list):
    file_list = [
        f"{data_path}output_tiff_scan2D_{sid}//detsum_{elem}_norm.tiff"
        for sid in sid_list
    ]
    im_stack = load_ims(file_list)
    im_stack_test = interp_sub_pix(im_stack, trans_matrix)
    imp = np.sum(im_stack_test, axis=0)
    stack.append(imp[np.newaxis, ...])

stack = np.concatenate(stack, axis=0)
#     tifffile.imwrite(''.join([data_path, 'original_rsm_100x100_nm//stack_aligned_', elem, '_100x100_nm.tif']),im_stack_test.astype(np.float32),imagej=True)

Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# sum all diffraction patterns from local files to determine roi and mask

tot = sum_all_h5_data_db_parallel(sid_list, det = 'merlin1')
plt.imshow(np.log10(tot+0.1),clim=[1,5])
plt.show()

In [ ]:
plt.imshow(np.log10(np.flip(tot,1)+0.1),clim=[1,5])
plt.show()

In [ ]:
%matplotlib inline
plt.imshow(np.log10(tot+0.1),clim=[1,5])
plt.show()

In [ ]:
stack.shape[0]

In [4]:
roi = [15,80,110,110]  # [slice(row start, end), slice(col start,end)]
# files = [get_path(sid,'merlin2') for sid in sid_list]
scan_row = im_row
scan_col = im_col
diff_data = load_h5_data_db_parallel(sid_list,det='merlin1',mon='sclr1_ch4',roi=roi)
# diff_data = nanorsm.load_h5_files_parallel(files,roi)
sz = diff_data.shape
diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)

Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
%matplotlib inline
sz = diff_data.shape
single_im = np.sum(diff_data.reshape(-1,sz[-2],sz[-1]),0)
plt.imshow(np.log10(single_im))

In [ ]:
# Load, normalize, align, and assemble diffraction data 
# diff_data is the 5D dataset aligned; make sure your machine has enough memory
'''
data_path = '//data//home//home//hyan//export//scan_180000//'
prefix = 'scan_'
postfix = '_merlin1.h5'
scan_row = 100
scan_col = 100

data_file_list = create_file_list(data_path, prefix, postfix, sid_list)
diff_data = load_h5_data(data_file_list,roi=roi,mask=mask)

postfix = '_merlin1.txt'
mon_file_list = create_file_list(data_path, prefix, postfix, sid_list)

mon_data = load_scaler_data(mon_file_list,['sclr1_ch4'])

# First scaler reading of each scan is zero; refill with first non-zero reading
avg = np.mean(mon_data[mon_data != 0])
mon_data[mon_data==0] = avg 

sz = diff_data.shape

diff_data = (diff_data/mon_data[:,np.newaxis,np.newaxis])

diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)
'''

In [5]:
### transform to cartesian crystal coordinates (z along hkl and x is the rocking direction)

energy = 12
delta = 19.5
gamma = 13.5
num_angle = 6
th_step = 0.5
pix = 55
det_dist = 5e5
offset = [0,0]
data_store = 'reduced' # this will reduce the data stored. If use 'full' it can be over 100G
# save_path = '//data//users//2024Q2//Yan_2024Q2//MoTe2//nanoRSM//'

method = {'fit_type': 'com',# fitting method: center of mass, 'com' or 'peak'
         'shape': 'gaussian', # peak shape: 'gaussian', 'lorentzian', or 'voigt'
         'n_peaks': [1,1,1]}, # number of peaks in each direction, qx, qy, and qz


# generate an object of the RSM class
rsm = RSM(diff_data,energy,delta,gamma,num_angle,th_step,pix,det_dist,offset,stack,elem_list)
# transform from detector coordinates to crystal coordinates
rsm.calcRSM('cryst',data_store,desired_workers=20)
# calculate strain
# 'com', center of mass, is a simple algorithm to calculate the strain. Note: There is an abitrary offset
rsm.calcSTRAIN(method)  
# show results
rsm.disp()
# save results
#rsm.save(save_path)
# also save the entire object
# save_file = ''.join([save_path,'rsm_35_blanket_film.obj'])
# pickle.dump(rsm, open(save_file,'wb'),protocol = 4)

Processing scans in parallel:   0%|          | 0/3600 [00:00<?, ?it/s]

raw det_data is deleted
qxz_data: [pos,qx,qz] with dimensions of (60, 60, 8, 668)
qyz_data: [pos,qy,qz] with dimensions of (60, 60, 109, 668)


  0%|          | 0/3600 [00:00<?, ?it/s]

In [11]:
method = {'fit_type': 'peak',
         'shape': 'gaussian',
         'n_peaks': [1,1,1]}
rsm.calcSTRAIN(method)
rsm.disp()

  0%|          | 0/3600 [00:00<?, ?it/s]

In [ ]:
plt.figure()
plt.imshow(np.squeeze(rsm.strain))
plt.show()

In [ ]:
np.zeros(5)

In [ ]:
plt.figure()
plt.imshow(np.log10(rsm.qxz_data[30,70,:,:].T+1e-7),clim=[-6,0])
plt.show()

In [ ]:
data_path = '//data//users//2020Q3//Ajith_2020Q3//LPE_FeS2_P4_diff//'
save_file = ''.join([data_path,'rsm.obj'])
rsm = pickle.load(open(save_file,'rb'))
rsm.disp()

In [ ]:
def interactive_map(
    names,
    im_stack,
    label,
    data_4D,
    cmap='jet',
    clim=None,
    marker_color='black'
):
    n = min(len(names), im_stack.shape[0])
    total = n + 1
    ncols, nrows = 3, int(np.ceil(total / 3))

    fig, axs = plt.subplots(
        nrows, ncols,
        figsize=(3 * ncols, 3 * nrows),
        gridspec_kw={'wspace': 0.1, 'hspace': 0.2}
    )
    axs = axs.ravel()

    # initial plotting
    for ax, name, img in zip(axs, names[:n], im_stack[:n]):
        ax.imshow(img, cmap=cmap, aspect='auto')
        ax.set_title(name, fontsize=9)
        ax.axis('off')

    diff_ax = axs[n]
    im = diff_ax.imshow(data_4D[0, 0], cmap=cmap, clim=clim, aspect='auto')
    diff_ax.set_title(label, fontsize=9)
    diff_ax.axis('off')

    for ax in axs[total:]:
        ax.axis('off')

    cbar = fig.colorbar(im, ax=diff_ax, fraction=0.046, pad=0.02)
    cbar.ax.tick_params(labelsize=8)

    # tighten BEFORE connecting
    fig.tight_layout()

    def onclick(event):
        if event.inaxes not in axs:
            return
        col, row = int(round(event.xdata)), int(round(event.ydata))
        if not (0 <= col < im_stack.shape[2] and 0 <= row < im_stack.shape[1]):
            return

        # redraw each panel
        for i, ax in enumerate(axs[:total]):
            ax.clear()
            if i < n:
                ax.imshow(im_stack[i], cmap=cmap, aspect='auto')
                ax.plot(col, row, 'o', color=marker_color, ms=5)
                ax.set_title(names[i], fontsize=9)
            else:
                ax.imshow(data_4D[row, col], cmap=cmap, clim=clim, aspect='auto')
                ax.set_title(label, fontsize=9)
            ax.axis('off')

        # immediate redraw
        fig.canvas.draw()

    # connect *after* layout
    fig.canvas.mpl_connect('button_press_event', onclick)

    fig.show()

    
def load_diff_data(sid, scaler_names, det_name, mon=None):
    """
    Fetch scan data and build:
      - im_stack: array of shape (len(scaler_names), scan_row, scan_col)
      - diff_data: array of shape (scan_row, scan_col, det_rows, det_cols)
    """
    h = db[sid]
    df = h.table()
    s = scan_command(sid).split()
    scan_col, scan_row = int(s[3]), int(s[7])

    # build scalar stacks
    stacks = []
    for name in scaler_names:
        if name in df:
            arr = np.array(df[name])
        else:
            arr = np.array(df['Det1_'+name] + df['Det2_'+name] + df['Det3_'+name])
        stacks.append(arr.reshape(scan_row, scan_col))
    im_stack = np.stack(stacks, axis=0)

    if mon is not None:
        mon_arr = np.array(df[mon]).reshape(1, scan_row, scan_col)
        im_stack = im_stack / mon_arr

    # load diffraction data and reshape
    diff = list(h.data(det_name))
    det = np.asarray(diff).reshape(scan_row, scan_col, *diff[0].shape)
    return im_stack, det


In [ ]:
%matplotlib notebook
interactive_map(elem_list,stack,'rsm',np.log10(np.swapaxes(rsm.qxz_data,2,3)),cmap='jet')

In [ ]:
plt.figure()
plt.imshow(np.log10(np.swapaxes(rsm.qxz_data,2,3)[11,25,:,:]+1e-7),cmap='jet',clim=[-4.8,-2.5])

In [ ]:
# interactive figure

%matplotlib qt

# load fluorescence image stacks
# insert your own code accordingly

sum_Ge = np.sum(Ge,0)
sum_W = np.sum(W,0)
sum_Cu = np.sum(Cu,0)

cmap = 'viridis'
color = 'red'
eps = 1e-5
def onclick(event):
    global row, col
    col, row = event.xdata, event.ydata
    if col is not None and row is not None:
        row = int(np.round(row))
        col = int(np.round(col))
        ax0.clear()
        ax0.imshow(sum_Ge,cmap=cmap)
        ax0.plot(col,row,marker='o',markersize=2, color=color)
        ax0.set_title('Ge')
        ax1.clear()
        ax1.imshow(sum_W,cmap=cmap)
        ax1.plot(col,row,marker='o',markersize=2, color=color)
        ax1.set_title('W')
        ax2.clear()
        ax2.imshow(sum_Cu,cmap=cmap)
        ax2.plot(col,row,marker='o',markersize=2, color=color)
        ax2.set_title('Cu')
        ax3.clear()
        ax3.imshow(rsm.tot,cmap=cmap)
        ax3.plot(col,row,marker='o',markersize=2, color=color)
        ax3.set_title('tot')
        ax4.clear()
        ax4.imshow(np.log10(rsm.qxz_data[row,col,:,:].T+eps),cmap=cmap)
        ax4.set_title('RSM')
        fig.canvas.draw_idle()
    
    return

fig = plt.figure()
spec = fig.add_gridspec(2,3)
#fig, ax = plt.subplots(2,1)
ax0 = fig.add_subplot(spec[0,0])
ax0.imshow(clean_Ge,cmap=cmap)
ax0.set_title('Ge')

ax1 = fig.add_subplot(spec[0,1])
ax1.imshow(sum_W,cmap=cmap)
ax1.set_title('W')

ax2 = fig.add_subplot(spec[0,2])
ax2.imshow(sum_Cu,cmap=cmap)
ax2.set_title('Cu')

ax3 = fig.add_subplot(spec[1,0])
ax3.imshow(rsm.tot,cmap=cmap)
ax3.set_title('tot')

ax4 = fig.add_subplot(spec[1,1:3])
ax4.imshow(np.log10(rsm.qxz_data[0,0,:,:].T+eps),cmap=cmap)
ax4.set_title('RSM')
plt.tight_layout()

# Call click func
cid = fig.canvas.mpl_connect('button_press_event', onclick)

#plt.show()

In [ ]:
# create movie
import matplotlib
import matplotlib.animation as manimation 
matplotlib.use('AGG') 
def update_fig(row,col):

    #col = col + i
    ax0.clear()
    ax0.imshow(sum_Ge,cmap=cmap)
    ax0.plot(col,row,marker='o',markersize=2, color=color)
    ax0.set_title('Ge')
    ax1.clear()
    ax1.imshow(sum_W,cmap=cmap)
    ax1.plot(col,row,marker='o',markersize=2, color=color)
    ax1.set_title('W')
    ax2.clear()
    ax2.imshow(sum_Cu,cmap=cmap)
    ax2.plot(col,row,marker='o',markersize=2, color=color)
    ax2.set_title('Cu')
    ax3.clear()
    ax3.imshow(rsm.tot,cmap=cmap)
    ax3.plot(col,row,marker='o',markersize=2, color=color)
    ax3.set_title('tot')
    ax4.clear()
    ax4.imshow(np.log10(rsm.qxz_data[row,col,:,:].T+eps),cmap=cmap)
    ax4.set_title('RSM')
    fig.canvas.draw_idle()
    return  
     
def init():
    fig = plt.figure()
    spec = fig.add_gridspec(2,3)
    
    ax0 = fig.add_subplot(spec[0,0])
    ax0.imshow(clean_Ge,cmap=cmap)
    ax0.set_title('Ge')

    ax1 = fig.add_subplot(spec[0,1])
    ax1.imshow(sum_W,cmap=cmap)
    ax1.set_title('W')

    ax2 = fig.add_subplot(spec[0,2])
    ax2.imshow(sum_Cu,cmap=cmap)
    ax2.set_title('Cu')

    ax3 = fig.add_subplot(spec[1,0])
    ax3.imshow(rsm.tot,cmap=cmap)
    ax3.set_title('tot')

    ax4 = fig.add_subplot(spec[1,1:3])
    ax4.imshow(np.log10(rsm.qxz_data[0,0,:,:].T+eps),cmap=cmap)
    ax4.set_title('RSM')
    plt.tight_layout()
    return  

FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(title='Movie', artist='hyan',
                comment='RSM variation across GAAFET')
writer = FFMpegWriter(fps=5, metadata=metadata)

init()

with writer.saving(fig, 'myfile.mp4', dpi=100):
    writer.grab_frame()
    row = 21
    col = 5
    for j in range(44):
        update_fig(row,col+j)
        writer.grab_frame()
    row = 2
    col = 28
    for j in range(40):
        update_fig(row+j,col)
        writer.grab_frame()
    #writer.finish()
 